## tf.Summary
- 用Web形式展示，便于分布式或跨平台应用
- Summary Ops:主要用于获取张量
    - audio
    - image
    - scalar
    - historgram
    - **merge_all**：将所有汇总操作汇聚在一起，生成一个汇聚操作 merged，是所有汇总操作的后置操作
- Filer Writer：用于向事件文件写入序列化数据，包括(P141)：**在P164有定义proto文件**
    - 汇总数据：一组将被可视化的具名数值，包括图像音频等
    - 事件数据：执行操作时产生的事件信息，像世间戳，全局步数等
    - 数据流图：当前会话加载的默认数据流图
- Summary：汇总数据
- Event：事件数据

## 可视化

### 节点
- 之前已经知道的有计算，存储，数据节点
- 引入抽象节点：一组特定操作的集合，在网络中通常会将功能相同的操作整合为一个抽象节点
- 关于name_scope和variable_scope的[区别](https://blog.csdn.net/u012436149/article/details/53081454?_t_t_t=0.5758701422798641)  
    - name_scope对 get_variable()创建的变量 的名字不会有任何影响,而创建的op或者变量会被加上前缀
    - 使用tf.get_variable()的时候，tf.name_scope()就不会给 tf.get_variable()创建出来的Variable加前缀。但是 tf.Variable() 创建出来的就会受到 name_scope 的影响.
- 实例：**将op和变量等定义到一层，下面这个例子中形成全连接层，比如再和可能有的非线性激活结合，再往上封装**

In [ ]:
#通过多层命名简化数据流图
with tf.variable_scope('layer_one', reuse=True):
    with tf.variable_scope('weight'):
        weight = tf.get_variable('w', [10, 2], tf.randon_normal_initializer())
    with tf.variable_scope('bias', reuse=True):
        bias = tf.get_variable('b', [1, 2], tf.constant_initializer())
    #这里通过多层定义使得网络更加整洁
    with tf.name_scope('Wx_plus_b'):
        y = tf.add(tf.matmul(x, weight), bias)

##### mnist数据集的实例

In [ ]:
import tensorflow as tf
from tensorflow.example.tutorials.mnist import input_data


mnist = input_data.read_data_sets('data/', one_hot=True)
#输入
with tf.name_scope('input'):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='X')
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Y')
#可视化
with tf.name_scope('reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 3])
    tf.summary.image('input', image_shaped_input, 10)
#网络
with tf.variable_scope('layer'):
    with tf.variable_scope('weight'):
        weight = tf.get_variable('weight', [784, 10], tf.random_normal_initializer())
        tf.summary.histogram('weight', weight)
    with tf.variable_scope('bias'):
        bias = tf.get_variable('bias', [10], tf.constant_initializer())
    with tf.name_scope('Wx_add_b'):
        y_ = tf.add(tf.matmal(x, weight), bias)
#cost
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropywith_logits(labels=Y, logits=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
        tf.summary.scalar('cross_entropy', cross_entropy)
#优化器
with tf.name_scope('train'):
    train_op = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
#评价指标
with tf.name_scope('accuracy'):
    with tf.name_scrope('predict'):
        prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(prediction), tf.float32)
        tf.summary.scalar('accuracy', accuracy)

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    writer = tf.Summary.FileWriter('./logs')
    sess.run(tf.global_variables_initilizer())
    for i in range(100):#max_step
        if i % 5 == 0:#summary_step
            summary, acc = sess.run([merged, accuracy], feed_dict={})
            #写入汇总数据，传入全局步数
            writer.add_summary(summary, i)

#### tf.summary.scalar()
- 获取带有一个标量值的汇总数据,比如loss，accuracy等
- 参数
    - name
    - tensor
    - collections = GraphKeys.SUMMARIES

#### tf.summary.histogram
- 数据统计信息
- 参数和scalar一样

#### tf.summary.image
- 获取带有图片的汇总操作
- 参数
    - name
    - tensor:必须是四阶张量
    - max_outputs：渲染的数量
    - collections

#### tf.summary,audio
- 音频
- 参数
    - name
    - tensor
    - sample_rate:采样率
    - max_outputs
    - collections
- [magenta](magenta.tensorflow.org)

[一个例子](https://blog.csdn.net/qq_31120801/article/details/75268765)
- 其中tf.RunOption()定义TensorFlow运行选项，其中设置trace_level为FULL_TRACE，并使用tf.RunMetadata()定义TensorFlow运行的元信息，这样可以记录训练时运算时间和内存占用等方面的信息。

### 图像嵌入表示